# Second virial coefficient calculation of SPC/E using Mayer-Sampling


Here, we reproduce the virial coefficient of SPCE

Table 2 of https://www.doi.org/10.1021/jp0710685

Figure 1 of https://doi.org/10.1063/1.5016165

In [1]:
import unittest
import feasst as fst

class TestMayerSamplingSPCE(unittest.TestCase):
    def test(self):
        mc = fst.MonteCarlo()
        config = fst.MakeConfiguration(fst.args({"cubic_box_length": str(fst.NEAR_INFINITY)}))
        config.add_particle_type(fst.install_dir() + "/forcefield/data.spce")
        config.add_particle_type(fst.install_dir() + "/forcefield/data.spce", "2")
        config.add_particle_of_type(0)
        config.add_particle_of_type(1)
        for site in range(config.num_site_types()):
            config.set_model_param("cutoff", site, 1e4)
        mc.add(config)
        two_body = fst.MakeModelTwoBodyFactory()
        two_body.add(fst.MakeLennardJones())
        two_body.add(fst.MakeCoulomb())
        mc.add(fst.MakePotential(two_body))
        mc.add_to_reference(fst.MakePotential(fst.MakeHardSphere()))
        mc.set(fst.MakeThermoParams(fst.args({"beta": str(1./fst.kelvin2kJpermol(773))})))
        mayer = fst.MakeMayerSampling()
        mc.set(mayer)
        mc.add(fst.MakeTrialTranslate(fst.args({"new_only": "true", "reference_index": "0",
            "tunable_param": "1", "particle_type": "1"})))
        mc.add(fst.MakeTrialRotate(fst.args({"new_only": "true", "reference_index": "0",
            "tunable_param": "40"})))
        steps_per = "1e5"
        mc.add(fst.MakeLogAndMovie(fst.args({"steps_per": steps_per, "file_name": "spce_b2"})))
        mc.add(fst.MakeCheckRigidBonds(fst.args({"steps_per": steps_per})))
        mc.add(fst.MakeTune(fst.args({"steps_per": steps_per})))
        mc.attempt(int(1e6))
        mc.perform(fst.MakeRemoveModify(fst.args({"name": "Tune"})))
        mc.attempt(int(1e7))
        mayer = fst.MakeMayerSampling()
        mc.set(mayer)
        mc.attempt(int(1e8))
        b2hs = 2./3.*fst.PI*mc.configuration().model_params().sigma().value(0)**3  # A^3
        b2hs *= 1e-30*1e3*mc.configuration().physical_constants().avogadro_constant() # L/mol
        print(b2hs*mayer.second_virial_ratio())
        self.assertAlmostEqual(-0.08596, b2hs*mayer.second_virial_ratio(), delta=0.006)

If the test passes, the energy is within the tolerance of the SRSW value and the two ensemble average methods agreed.

In [2]:
%time  # Note: any line starting with % is only to be used with ipynb
unittest.main(argv=[''], verbosity=2, exit=False)

test (__main__.TestMayerSamplingSPCE) ... 

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs
-0.08526363929289871


ok

----------------------------------------------------------------------
Ran 1 test in 528.655s

OK


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!